In [261]:
from time import process_time_ns
from types import new_class
from typing import List
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(suppress=True)
movies = pd.read_csv('movies.csv', low_memory=False)
ratings = pd.read_csv('ratings.csv', low_memory=False)
asc = ratings.sort_values(by=['userId', 'timestamp'])
# print(asc)
# print(asc.values)
def getListUid(uid):
    ratings_temp = ratings
    movies_temp = movies
    
    final_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='rating')
    time_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='timestamp')
    # print(time_dataset.head(10))
    desc = ratings_temp[ratings_temp['userId'] == uid].sort_values(by='timestamp', ascending=False)
    arr =desc.head(5).values
    # print(arr)
    movie_list = arr[:,1]
    # movie_list = np.append(movie_list,16)
    # print(movie_list)
    # print(ratings_temp)
    # a= ratings_temp.loc[movie_list]
    # print(a)
    # print(final_dataset)

    final_dataset.fillna(0,inplace=True)
    time_dataset.fillna(0,inplace=True)
    # time_dataset.fillna(0,inplace=True)
    # print(final_dataset.head())
    no_user_voted = ratings_temp.groupby('movieId')['rating'].agg('count')
    # print(no_user_voted)
    no_movies_voted = ratings_temp.groupby('userId')['rating'].agg('count')
    final_dataset = final_dataset.loc[no_user_voted[no_user_voted > 10].index,:]
    final_dataset=final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
    # print(final_dataset)
    final_dataset = final_dataset.loc[movie_list]

    final_dataset = final_dataset.T
    # print(final_dataset.iat[56])
    new_arr=  np.array(final_dataset)
    list_data = new_arr.tolist()
    knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
    knn.fit(list_data)
    distances , indices =knn.kneighbors([list_data[0]],n_neighbors=10)
    rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
    # # print(rec_movie_indices)
    list_uid = list(zip(*rec_movie_indices))[0]
    return sorted(zip(distances.tolist()[0], indices.tolist()[0]))
    # return indices.tolist()[0]


In [262]:
def get_recommendation(uid,movie):
    list_uid = getListUid(uid)
    list_uid.pop(0)
    ratings_temp = ratings
    movies_temp = movies
    
    final_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='rating')
    time_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='timestamp')
    test_id = ratings_temp[ratings_temp['userId'] == 56]
    # print(test_id)
    # print()
    x = int(test_id.size/4)
    count_phim = []
    for i in range(x):
        if (test_id.iloc[i].at['rating'] >= 4):
            count_phim.append(test_id.iloc[i].at['movieId'])
    # print(count_phim)
    
    # print(test_id.iloc[1].at['rating'])
    # for i in test_id.rows:
    #     print(i['movieId'])
    count_phim = []
    # total = []
    np.set_printoptions(suppress=True,\
    formatter={'float_kind':'{:0.0f}'.format})
    for i in list_uid:
        # currId = ratings_temp[ratings_temp['userId'] == i[1]]
        # print(i[1])
        test_id = ratings_temp[ratings_temp['userId'] == i[1]]
        # print(test_id)
        # print()
        x = int(test_id.size/4)
        for j in range(x):
            if (test_id.iloc[j].at['rating'] >= 4):
                count_phim.append(test_id.iloc[j].at['movieId'])
    
    unique, counts = np.unique(np.array(count_phim), return_counts=True)
    final_arr = np.asarray((unique, counts)).T.tolist()
    
    print (sorted(final_arr,key=lambda l:l[1], reverse=True))
    
    

get_recommendation(43,"Iron Man")

[[296.0, 4.0], [608.0, 4.0], [1.0, 3.0], [47.0, 3.0], [50.0, 3.0], [249.0, 3.0], [350.0, 3.0], [480.0, 3.0], [527.0, 3.0], [589.0, 3.0], [593.0, 3.0], [1136.0, 3.0], [1193.0, 3.0], [1206.0, 3.0], [2762.0, 3.0], [2804.0, 3.0], [2858.0, 3.0], [2959.0, 3.0], [4306.0, 3.0], [4886.0, 3.0], [4979.0, 3.0], [6874.0, 3.0], [6.0, 2.0], [29.0, 2.0], [32.0, 2.0], [36.0, 2.0], [95.0, 2.0], [111.0, 2.0], [161.0, 2.0], [215.0, 2.0], [260.0, 2.0], [292.0, 2.0], [293.0, 2.0], [318.0, 2.0], [329.0, 2.0], [356.0, 2.0], [364.0, 2.0], [377.0, 2.0], [434.0, 2.0], [457.0, 2.0], [474.0, 2.0], [494.0, 2.0], [541.0, 2.0], [586.0, 2.0], [587.0, 2.0], [595.0, 2.0], [596.0, 2.0], [597.0, 2.0], [858.0, 2.0], [1036.0, 2.0], [1127.0, 2.0], [1196.0, 2.0], [1198.0, 2.0], [1200.0, 2.0], [1210.0, 2.0], [1213.0, 2.0], [1214.0, 2.0], [1221.0, 2.0], [1225.0, 2.0], [1240.0, 2.0], [1259.0, 2.0], [1265.0, 2.0], [1270.0, 2.0], [1291.0, 2.0], [1517.0, 2.0], [1527.0, 2.0], [1580.0, 2.0], [1641.0, 2.0], [1719.0, 2.0], [1923.0, 2.0